In [1]:
import numpy as np
import cv2
import os
from keras.layers import *
from keras.models import Model
from unet import unet
import keras.backend as K

Load all trained models.

In [4]:
char_list = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789'
#CRNN model
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
model_crnn = Model(inputs, outputs)

In [5]:
# Insert your path to weights file
model_lines = unet(pretrained_weights='/content/drive/MyDrive/PageSegData/unet_lines.h5')

model_words = unet(pretrained_weights='/content/drive/MyDrive/PageSegData/unet_words.h5')

model_crnn.load_weights('/content/drive/MyDrive/PageSegData/best_model.hdf5')

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Testing with generated data:

In [85]:
!python generate_data.py --n_samples 5 --word_type uppercase

In [56]:
from pipeline_utils import preprocess_img

def test_images(path, i, j):
  imgs = []
  for i in range(i, j):
    img = cv2.imread(os.path.join(path, str(i) + '.jpg'), 0)
    img=preprocess_img(img,(128,32))
    img=np.expand_dims(img,axis=-1)
    img = img/255.
    img=np.expand_dims(img,axis=0)
    imgs.append(img)
    return imgs

def test_image(path, i):
  img = cv2.imread(os.path.join(path, str(i) + '.jpg'), 0)
  img=preprocess_img(img,(128,32))
  img=np.expand_dims(img,axis=-1)
  img = img/255.
  img=np.expand_dims(img,axis=0)
  return img

In [88]:
i = 5  # number of testing images (from data generator)

# predict outputs on validation images
for i in range(i):
  prediction = model_crnn.predict(test_image('/content/images/', i))
  
  # use CTC decoder
  out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                          greedy=True)[0][0])
  
  # see the results
  with open('annotation.txt', 'r+') as file:
      lines = file.readlines()
      print("original_text =  ", lines[i].split(',')[1])
      print("predicted text = ", end = '')
      for p in out[0]:  
          if int(p) != -1:
              print(char_list[int(p)], end = '') 
      print('\n')      

1/1 [==============================] - 0s 63ms/step
original_text =   И7ИЕСЙЦ

predicted text = И7ИЕСЙЦ

1/1 [==============================] - 0s 67ms/step
original_text =   ЫЙЩЭ87ЛХ26ДЮ

predicted text = ЫЙЩЭ87ЛХ26ДЮ

1/1 [==============================] - 0s 69ms/step
original_text =   408УЛН3ЭВНЛЦ9ЯЗ

predicted text = 408УЛН3ЭВНЛЦ9ЯЗ

1/1 [==============================] - 0s 69ms/step
original_text =   ЦЧДЛВ

predicted text = ЦЧДЛВ

1/1 [==============================] - 0s 69ms/step
original_text =   61Д

predicted text = 61Д



**Training is over. Recognition pipeline will be presented in the pipeline.py**